# Organize ML projects with Scikit-Learn

While Machine Learning is powerful, people often overestimate it: apply machine learning to your project, and all your problems will be solved. In reality, it's not this simple. To be effective, one needs to organize the work very well. In this notebook, we will walkthrough practical aspects of a ML project. To look at the big picture, let's start with a checklist below. It should work reasonably well for most ML projects, but make sure to adapt it to your needs:

1. **Define the scope of work and objective**
    * How is your solution be used?
    * How should performance be measured? Are there any contraints?
    * How would the problem be solved manually?
    * List the available assumptions, and verify if possible.
    
    
2. **Get the data**
    * Document where you can get that data
    * Store data in a workspace you can easily access
    * Convert the data to a format you can easily manipulate
    * Check the overview (size, type, sample, description, statistics)
    * Data cleaning
    
    
3. **EDA & Data transformation**
    * Study each attribute and its characteristics (missing values, type of distribution, usefulness)
    * Visualize the data
    * Study the correlations between attributes
    * Feature selection, Feature Engineering, Feature scaling
    * Write functions for all data transformations
    
    
4. **Train models**
    * Automate as much as possible
    * Train promising models quickly using standard parameters. Measure and compare their performance
    * Analyze the errors the models make
    * Shortlist the top three of five most promising models, preferring models that make different types of errors.


5. **Fine-tunning**
    * Treat data transformation choices as hyperparameters, expecially when you are not sure about them (e.g., replace missing values with zeros or with the median value)
    * Unless there are very few hyperparameter value to explore, prefer random search over grid search.
    * Try ensemble methods
    * Test your final model on the test set to estimate the generalizaiton error. Don't tweak your model again, you would start overfitting the test set.

## Example: Articles categorization

### Objectives

Build a model to determine the categories of articles. 

### Get Data

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

In [3]:
bbc = pd.read_csv('https://raw.githubusercontent.com/dhminh1024/practice_datasets/master/bbc-text.csv')

In [4]:
bbc.sample(5)

,category,text
2026,politics,job cuts false economy - tuc plans to shed ...
240,business,retail sales show festive fervour uk retail sa...
1778,business,water firm suez in argentina row a conflict be...
1187,sport,newcastle to join morientes race newcastle hav...
1053,entertainment,black sabbath top rock album poll black sabbat...


In [5]:
bbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [6]:
from sklearn.model_selection import train_test_split

X = bbc['text']
y = bbc['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import regex as re
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
stop_words = stopwords.words('english')
def preprocessor(text):
    text = (re.sub('[\W]+', ' ', text.lower()))
    return text

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [36]:
tfidf = TfidfVectorizer(stop_words = stop_words,
                        tokenizer = tokenizer_porter,
                        preprocessor = preprocessor)

In [39]:
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=42))])
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test,predictions))
print('Classification report:\n', classification_report(y_test,predictions))

Accuracy: 0.9760479041916168
Classification report:
                precision    recall  f1-score   support

     business       0.95      0.97      0.96       164
entertainment       1.00      0.96      0.98       113
     politics       0.96      0.97      0.97       113
        sport       0.99      1.00      0.99       146
         tech       0.98      0.97      0.98       132

     accuracy                           0.98       668
    macro avg       0.98      0.98      0.98       668
 weighted avg       0.98      0.98      0.98       668



In [50]:
clf = Pipeline([('vect', tfidf),
                ('clf', LinearSVC(random_state=42))])
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test,predictions))
print('Classification report:\n', classification_report(y_test,predictions))

Accuracy: 0.9760479041916168
Classification report:
                precision    recall  f1-score   support

     business       0.97      0.94      0.96       164
entertainment       0.97      0.98      0.98       113
     politics       0.96      0.98      0.97       113
        sport       0.99      1.00      0.99       146
         tech       0.98      0.98      0.98       132

     accuracy                           0.98       668
    macro avg       0.98      0.98      0.98       668
 weighted avg       0.98      0.98      0.98       668



In [41]:
clf = Pipeline([('vect', tfidf),
                ('clf', DecisionTreeClassifier())])
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test,predictions))
print('Classification report:\n', classification_report(y_test,predictions))

Accuracy: 0.8323353293413174
Classification report:
                precision    recall  f1-score   support

     business       0.82      0.78      0.80       164
entertainment       0.84      0.81      0.82       113
     politics       0.77      0.82      0.79       113
        sport       0.87      0.90      0.89       146
         tech       0.86      0.86      0.86       132

     accuracy                           0.83       668
    macro avg       0.83      0.83      0.83       668
 weighted avg       0.83      0.83      0.83       668



In [51]:
from sklearn.ensemble import VotingClassifier

base_classifiers = [('LogisticRegression', LogisticRegression(random_state=42)),
                    ('LinearSVC', LinearSVC(random_state=42)),
                    ('DecisionTree', DecisionTreeClassifier())]

clf_vote = VotingClassifier(estimators=base_classifiers, 
                            voting='soft', weights = [2,3,1]) 
clf_vote = Pipeline([('vect', tfidf),
                ('clf', clf_vote)])
clf_vote.fit(X_train, y_train)
print('Accuracy:', accuracy_score(y_test,predictions))
print('Classification report:\n', classification_report(y_test,predictions))

Accuracy: 0.9760479041916168
Classification report:
                precision    recall  f1-score   support

     business       0.97      0.94      0.96       164
entertainment       0.97      0.98      0.98       113
     politics       0.96      0.98      0.97       113
        sport       0.99      1.00      0.99       146
         tech       0.98      0.98      0.98       132

     accuracy                           0.98       668
    macro avg       0.98      0.98      0.98       668
 weighted avg       0.98      0.98      0.98       668

